In [1]:
%load_ext autoreload

In [48]:
%autoreload
from dotenv import load_dotenv
from src.data import scrape_genius_lyrics as scr
import os
import json

In [22]:
load_dotenv()
GENIUS = os.getenv("GENIUS")

## Sample pull of single song lyric

In [4]:
hit = scr.return_top_hit(['kaskade', 'angel on my shoulders'], token=GENIUS)
hit_url = scr.extract_url(hit)
hit_lyrics = scr.extract_lyrics(hit_url)

In [5]:
print(hit_lyrics)



[Verse 1]
It's time you see
The love you give
Is killing me
And I want to live
This isn't right
Search your soul for light

[Pre-Chorus]
So I'm turning left
I told myself
I could be as bad as someone else
But I find it hard
When angel's on my right

[Chorus]
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand

[Verse 2]
Why won't you go
No means no
Your dirty life
Is like a one man show
You're a night attack
You turn the light to black

[Pre-Chorus]
So I'm turning left
I told myself
I could be as bad as someone else
But I find it hard
When angel's on my right

[Chorus]
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand

[Bridge]
Talk to me
Walk with

## Use MSD to build search terms

Sample run with just 5 tracks

In [38]:
import pandas as pd
ut = '../data/external/MillionSongSubset/AdditionalFiles/subset_unique_tracks.txt'
df = pd.read_csv(ut, sep='<SEP>', header=None, engine='python',
                 names=['trackid', 'songid', 'artist', 'title'])
df.head()

,trackid,songid,artist,title
0,TRAAAAW128F429D538,SOMZWCG12A8C13C480,Casual,I Didn't Mean To
1,TRAAABD128F429CF47,SOCIWDW12A8C13D406,The Box Tops,Soul Deep
2,TRAAADZ128F9348C2E,SOXVLOJ12AB0189215,Sonora Santanera,Amor De Cabaret
3,TRAAAEF128F4273421,SONHOTT12A8C13493C,Adam Ant,Something Girls
4,TRAAAFD128F92F423A,SOFSOCN12A8C143F5D,Gob,Face the Ashes


Extract MSD track id and search terms based on arist and title

In [28]:
songs = []
for row in df.head().itertuples(index=False):
    tup = (row.trackid, [row.artist.lower(), row.title.lower()])
    songs.append(tup)
songs

[('TRAAAAW128F429D538', ['casual', "i didn't mean to"]),
 ('TRAAABD128F429CF47', ['the box tops', 'soul deep']),
 ('TRAAADZ128F9348C2E', ['sonora santanera', 'amor de cabaret']),
 ('TRAAAEF128F4273421', ['adam ant', 'something girls']),
 ('TRAAAFD128F92F423A', ['gob', 'face the ashes'])]

Use Genius search API to retrieve top hit for search term and store response with all meta data

In [43]:
genius = []
for song in songs:
    trackid, search_terms = song
    hit = scr.return_top_hit(search_terms, token=GENIUS)
    tup = [trackid, hit]
    genius.append(tup)
    
# serialize
with open('../data/interim/genius_metadata.json', 'w') as outfile:
    json.dump(genius, outfile)

[['TRAAAAW128F429D538',
  {'highlights': [],
   'index': 'song',
   'result': {'annotation_count': 14,
    'api_path': '/songs/13078',
    'full_title': "I Didn't Mean To by\xa0Casual (Ft.\xa0Del\xa0The Funky Homosapien)",
    'header_image_thumbnail_url': 'https://images.genius.com/6e1e86e7d9fe4b305b19612ea7ac9355.300x298x1.jpg',
    'header_image_url': 'https://images.genius.com/6e1e86e7d9fe4b305b19612ea7ac9355.400x397x1.jpg',
    'id': 13078,
    'lyrics_owner_id': 50,
    'lyrics_state': 'complete',
    'path': '/Casual-i-didnt-mean-to-lyrics',
    'primary_artist': {'api_path': '/artists/2904',
     'header_image_url': 'https://images.genius.com/14b78c83e54cc4084facb840e8387e22.1000x750x1.jpg',
     'id': 2904,
     'image_url': 'https://images.genius.com/14b78c83e54cc4084facb840e8387e22.1000x750x1.jpg',
     'is_meme_verified': False,
     'is_verified': False,
     'name': 'Casual',
     'url': 'https://genius.com/artists/Casual'},
    'pyongs_count': None,
    'song_art_image_t

Use URL from genius search API to retrieve lyrics (no API token needed for this request)

In [49]:
with open('../data/interim/genius_metadata.json', 'r') as f:
    genius = json.load(f)

lyrics = []
for song in genius:
    trackid = song[0]
    track_url = scr.extract_url(song[1])
    track_lyrics = scr.extract_lyrics(track_url)
    tup = [trackid, track_lyrics]
    lyrics.append(tup)

# serialize
with open('../data/interim/genius_lyrics.json', 'w') as outfile:
    json.dump(lyrics, outfile)